# AILib Tutorial 13: Tracing and Debugging

Learn how to debug and monitor your AI applications with AILib's comprehensive tracing system:

- Automatic tracing for agents and chains
- Manual trace instrumentation
- Trace visualization and analysis
- Performance monitoring
- Integration with observability tools

## Setup

In [ ]:
from ailib.tracing import (
    get_trace_manager, 
    start_trace, 
    end_trace,
    trace_step,
    get_current_trace
)
from ailib import create_agent, create_chain, tool
import json
from datetime import datetime
import time

## Automatic Tracing

In [ ]:
# Agents and chains are automatically traced

# Create a simple agent with tools
@tool
def calculate(expression: str) -> float:
    """Calculate a mathematical expression."""
    return eval(expression)

@tool 
def get_time() -> str:
    """Get the current time."""
    return datetime.now().strftime("%H:%M:%S")

# Create agent - tracing is automatic!
agent = create_agent(
    "math_assistant",
    tools=[calculate, get_time],
    verbose=True  # This also shows trace info
)

# Run agent - this creates a trace
result = agent.run("What time is it? Also calculate 25 * 4")
print(f"\nResult: {result}")

# Get the trace
manager = get_trace_manager()
traces = manager.list_traces()
print(f"\nTotal traces: {len(traces)}")

# Get the latest trace
if traces:
    latest_trace_id = traces[0]
    trace = manager.get_trace(latest_trace_id)
    print(f"\nLatest trace ID: {trace.trace_id}")
    print(f"Duration: {trace.duration_ms:.2f}ms")
    print(f"Steps: {len(trace.steps)}")

## Exploring Trace Data

In [ ]:
# Let's examine the trace in detail
if trace:
    print("=== Trace Details ===")
    print(f"Operation: {trace.operation_name}")
    print(f"Started: {trace.start_time}")
    print(f"Status: {trace.status}")
    print(f"\nMetadata: {json.dumps(trace.metadata, indent=2)}")
    
    print("\n=== Steps ===")
    for i, step in enumerate(trace.steps):
        print(f"\nStep {i+1}: {step.name}")
        print(f"  Type: {step.step_type}")
        print(f"  Duration: {step.duration_ms:.2f}ms")
        if step.metadata:
            print(f"  Metadata: {json.dumps(step.metadata, indent=4)}")
        if step.error:
            print(f"  ❌ Error: {step.error}")

## Manual Tracing

In [ ]:
# You can add custom traces to your code

def process_data_with_tracing(data: list) -> dict:
    """Process data with manual tracing."""
    
    # Start a new trace
    trace_id = start_trace(
        "data_processing",
        input_size=len(data),
        data_type="numbers"
    )
    
    try:
        # Step 1: Validation
        with trace_step("validation", step_type="validate"):
            if not data:
                raise ValueError("Empty data")
            if not all(isinstance(x, (int, float)) for x in data):
                raise ValueError("Invalid data type")
            time.sleep(0.1)  # Simulate work
        
        # Step 2: Analysis
        with trace_step("analysis", step_type="compute") as step:
            result = {
                "count": len(data),
                "sum": sum(data),
                "average": sum(data) / len(data),
                "min": min(data),
                "max": max(data)
            }
            step.metadata["result"] = result
            time.sleep(0.2)  # Simulate work
        
        # Step 3: Formatting
        with trace_step("formatting", step_type="format"):
            formatted = {
                "summary": f"Analyzed {len(data)} numbers",
                "stats": result,
                "timestamp": datetime.now().isoformat()
            }
            time.sleep(0.05)  # Simulate work
        
        end_trace(trace_id, status="success")
        return formatted
        
    except Exception as e:
        end_trace(trace_id, status="error", error=str(e))
        raise

# Test with tracing
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = process_data_with_tracing(data)
print("Processing result:")
print(json.dumps(result, indent=2))

# Get and examine the trace
latest_traces = manager.list_traces(limit=1)
if latest_traces:
    trace = manager.get_trace(latest_traces[0])
    print(f"\nTrace summary:")
    print(f"- Total duration: {trace.duration_ms:.2f}ms")
    for step in trace.steps:
        print(f"- {step.name}: {step.duration_ms:.2f}ms")

## Tracing Chains

In [ ]:
# Chains are also automatically traced

# Create a multi-step chain
analysis_chain = create_chain(
    "You are a data analyst. Analyze this data: {data}",
    "Summarize the key findings in bullet points",
    "What actions would you recommend based on this analysis?",
    verbose=True
)

# Run the chain
chain_result = analysis_chain.run(
    data="Sales increased 15% but customer satisfaction dropped 5%"
)

# Get the chain trace
chain_traces = manager.list_traces(limit=1)
if chain_traces:
    chain_trace = manager.get_trace(chain_traces[0])
    print("\n=== Chain Trace ===")
    print(f"Operation: {chain_trace.operation_name}")
    print(f"Total steps: {len(chain_trace.steps)}")
    print(f"Total duration: {chain_trace.duration_ms:.2f}ms")
    
    # Show each chain step
    for step in chain_trace.steps:
        if step.step_type == "chain_step":
            print(f"\n- {step.name}")
            print(f"  Duration: {step.duration_ms:.2f}ms")
            if 'step_name' in step.metadata:
                print(f"  Step: {step.metadata['step_name']}")

## Performance Analysis

In [ ]:
# Analyze performance across multiple runs

def run_performance_test(n_runs=5):
    """Run multiple operations and analyze performance."""
    
    agent = create_agent(
        "test_agent",
        tools=[calculate, get_time]
    )
    
    queries = [
        "Calculate 10 + 20",
        "What time is it?",
        "Calculate 100 * 25 / 5",
        "Get the current time and calculate 50 - 30"
    ]
    
    trace_ids = []
    
    for i in range(n_runs):
        query = queries[i % len(queries)]
        print(f"Run {i+1}: {query}")
        
        # Run and collect trace
        result = agent.run(query)
        traces = manager.list_traces(limit=1)
        if traces:
            trace_ids.append(traces[0])
    
    # Analyze performance
    durations = []
    step_counts = []
    
    for trace_id in trace_ids:
        trace = manager.get_trace(trace_id)
        if trace:
            durations.append(trace.duration_ms)
            step_counts.append(len(trace.steps))
    
    if durations:
        print("\n=== Performance Summary ===")
        print(f"Average duration: {sum(durations)/len(durations):.2f}ms")
        print(f"Min duration: {min(durations):.2f}ms")
        print(f"Max duration: {max(durations):.2f}ms")
        print(f"Average steps: {sum(step_counts)/len(step_counts):.1f}")

# Run performance test
run_performance_test()

## Trace Filtering and Search

In [ ]:
# Search and filter traces

# Get all recent traces
all_traces = manager.list_traces(limit=20)
print(f"Total traces available: {len(all_traces)}")

# Filter traces by operation
agent_traces = []
chain_traces = []
custom_traces = []

for trace_id in all_traces:
    trace = manager.get_trace(trace_id)
    if trace:
        if trace.operation_name.startswith("agent_"):
            agent_traces.append(trace)
        elif trace.operation_name.startswith("chain_"):
            chain_traces.append(trace)
        else:
            custom_traces.append(trace)

print(f"\nAgent traces: {len(agent_traces)}")
print(f"Chain traces: {len(chain_traces)}")
print(f"Custom traces: {len(custom_traces)}")

# Find slowest operations
if all_traces:
    traces_with_duration = []
    for trace_id in all_traces[:10]:  # Check last 10
        trace = manager.get_trace(trace_id)
        if trace and trace.duration_ms:
            traces_with_duration.append(trace)
    
    if traces_with_duration:
        slowest = max(traces_with_duration, key=lambda t: t.duration_ms)
        print(f"\nSlowest operation: {slowest.operation_name}")
        print(f"Duration: {slowest.duration_ms:.2f}ms")
        print(f"Steps: {len(slowest.steps)}")

## Error Tracing

In [ ]:
# Traces capture errors for debugging

@tool
def risky_operation(value: int) -> str:
    """A tool that might fail."""
    if value < 0:
        raise ValueError("Negative values not allowed")
    if value > 100:
        raise ValueError("Value too large")
    return f"Processed: {value}"

# Create agent with risky tool
risky_agent = create_agent(
    "risky_agent",
    tools=[risky_operation],
    max_steps=5
)

# Try operations that will fail
test_values = [50, -10, 150]

for value in test_values:
    print(f"\nTesting with value: {value}")
    try:
        result = risky_agent.run(f"Process the value {value}")
        print(f"Success: {result}")
    except Exception as e:
        print(f"Failed: {e}")
    
    # Check the trace
    latest = manager.list_traces(limit=1)
    if latest:
        trace = manager.get_trace(latest[0])
        if trace and trace.error:
            print(f"Trace captured error: {trace.error}")
            # Find which step failed
            for step in trace.steps:
                if step.error:
                    print(f"Failed at step '{step.name}': {step.error}")

## Custom Trace Visualization

In [ ]:
def visualize_trace(trace_id: str):
    """Create a simple text visualization of a trace."""
    trace = manager.get_trace(trace_id)
    if not trace:
        print("Trace not found")
        return
    
    print(f"\n{'='*60}")
    print(f"TRACE: {trace.operation_name}")
    print(f"ID: {trace.trace_id}")
    print(f"Duration: {trace.duration_ms:.2f}ms")
    print(f"Status: {trace.status}")
    print(f"{'='*60}")
    
    if trace.steps:
        # Calculate relative timings
        total_ms = trace.duration_ms
        
        for i, step in enumerate(trace.steps):
            # Calculate percentage of total time
            if total_ms > 0:
                percentage = (step.duration_ms / total_ms) * 100
            else:
                percentage = 0
            
            # Create visual bar
            bar_length = int(percentage / 2)  # Scale to 50 chars max
            bar = '█' * bar_length
            
            print(f"\n{i+1}. {step.name}")
            print(f"   Type: {step.step_type}")
            print(f"   Time: {step.duration_ms:.2f}ms ({percentage:.1f}%)")
            print(f"   [{bar:<50}]")
            
            if step.error:
                print(f"   ❌ ERROR: {step.error}")
    
    print(f"\n{'='*60}\n")

# Visualize the last few traces
recent_traces = manager.list_traces(limit=3)
for trace_id in recent_traces:
    visualize_trace(trace_id)

## Exporting Traces

In [ ]:
# Export traces for external analysis

def export_traces_to_json(trace_ids: list, filename: str):
    """Export traces to JSON for analysis."""
    traces_data = []
    
    for trace_id in trace_ids:
        trace = manager.get_trace(trace_id)
        if trace:
            traces_data.append(trace.to_dict())
    
    # Save to file (or return for display)
    return json.dumps(traces_data, indent=2, default=str)

# Export recent traces
recent = manager.list_traces(limit=5)
if recent:
    export_data = export_traces_to_json(recent[:2], "traces.json")
    print("Exported trace data (first 500 chars):")
    print(export_data[:500] + "...")

# You can also get traces as structured data
if recent:
    trace = manager.get_trace(recent[0])
    trace_dict = trace.to_dict()
    
    print("\n\nTrace structure:")
    print(f"- trace_id: {trace_dict['trace_id']}")
    print(f"- operation_name: {trace_dict['operation_name']}")
    print(f"- start_time: {trace_dict['start_time']}")
    print(f"- duration_ms: {trace_dict['duration_ms']}")
    print(f"- steps: {len(trace_dict['steps'])} items")

## Best Practices for Tracing

In [ ]:
# 1. Use meaningful operation names
def good_tracing_example():
    # Good: Descriptive operation name
    trace_id = start_trace(
        "user_registration_flow",
        user_type="premium",
        source="web"
    )
    
    # Good: Clear step names and types
    with trace_step("validate_email", step_type="validation"):
        pass
    
    with trace_step("create_account", step_type="database"):
        pass
    
    with trace_step("send_welcome_email", step_type="notification"):
        pass
    
    end_trace(trace_id, status="success")

# 2. Add relevant metadata
def trace_with_context(user_id: str, action: str):
    trace_id = start_trace(
        f"user_action_{action}",
        user_id=user_id,
        action=action,
        timestamp=datetime.now().isoformat()
    )
    
    try:
        with trace_step("process_action") as step:
            # Add step-specific metadata
            step.metadata["user_id"] = user_id
            step.metadata["action_type"] = action
            
            # Process...
            result = {"status": "completed"}
            step.metadata["result"] = result
        
        end_trace(trace_id, status="success", result=result)
        
    except Exception as e:
        end_trace(trace_id, status="error", error=str(e))
        raise

# 3. Use trace context in complex flows
def complex_workflow():
    # Parent trace
    parent_trace = start_trace("complex_workflow")
    
    try:
        # Sub-operation 1
        with trace_step("data_preparation", step_type="prep"):
            # More granular tracing within
            with trace_step("load_data", step_type="io"):
                pass
            with trace_step("validate_data", step_type="validation"):
                pass
        
        # Sub-operation 2  
        with trace_step("processing", step_type="compute"):
            with trace_step("algorithm_1", step_type="algorithm"):
                pass
            with trace_step("algorithm_2", step_type="algorithm"):
                pass
        
        end_trace(parent_trace, status="success")
        
    except Exception as e:
        end_trace(parent_trace, status="error", error=str(e))
        raise

print("✅ Best practices demonstrated")
print("\nKey principles:")
print("1. Use descriptive names for operations and steps")
print("2. Add relevant metadata for debugging")
print("3. Always close traces (use try/finally or context managers)")
print("4. Use consistent step types across your application")
print("5. Don't over-trace - focus on important operations")

## Summary

AILib's tracing system helps you understand and debug your AI applications:

- ✅ **Automatic tracing** for agents and chains
- ✅ **Manual instrumentation** for custom operations  
- ✅ **Rich metadata** capture for debugging
- ✅ **Performance analysis** to find bottlenecks
- ✅ **Error tracking** to diagnose failures
- ✅ **Export capabilities** for external analysis

Key benefits:
1. **Visibility** - See exactly what your AI is doing
2. **Debugging** - Quickly identify where things go wrong
3. **Performance** - Find and fix slow operations
4. **Monitoring** - Track application behavior over time
5. **Compliance** - Audit AI decision-making processes

## Next Steps

- Build production applications with comprehensive tracing
- Integrate with monitoring tools (Datadog, New Relic, etc.)
- Create custom dashboards for trace analysis
- Set up alerts based on trace patterns

Happy debugging! 🔍